# “Cafebrería” placement in México - Notebook

### Data recopilation

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


In [2]:
#Requirements for JSON files
CLIENT_ID = 'LMNU0FZBUMWDXBEOHY1CMBP3MZV33LVRKRZC5HYP3OMX1CYF' 
CLIENT_SECRET = 'QF4XWCLGG2N32G5FFBTP0HLWT0JRZDH3GA02VBRWDDNFXX3G' 
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: LMNU0FZBUMWDXBEOHY1CMBP3MZV33LVRKRZC5HYP3OMX1CYF
CLIENT_SECRET:QF4XWCLGG2N32G5FFBTP0HLWT0JRZDH3GA02VBRWDDNFXX3G


In [3]:
#From Mexico city we obtain the center of the city nd mark a big raddius of 15km to get the biggest amount of places
#Then we transform it to coordenates
address = 'Plaza de la Constitución S/N, Ciudad de México, CDMX'
geolocator = Nominatim(user_agent = "foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

19.4328333 -99.13316022232087


In [4]:
#Determine the parameters of search to put in the API request
search_query = 'Cafebrería'
radius = 15000
print(search_query + ' .... OK!')

Cafebrería .... OK!


In [5]:
#JSON request
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=LMNU0FZBUMWDXBEOHY1CMBP3MZV33LVRKRZC5HYP3OMX1CYF&client_secret=QF4XWCLGG2N32G5FFBTP0HLWT0JRZDH3GA02VBRWDDNFXX3G&ll=19.4328333,-99.13316022232087&v=20180604&query=Cafebrería&radius=15000&limit=30'

In [6]:
#Get the JSON response
results = requests.get(url).json()

In [7]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",False,4d482eabea3f3704dd06280a,Álvaro Obregón 86,MX,Cuauhtemoc,México,Córdoba,3130,"[Álvaro Obregón 86 (Córdoba), 06700 Cuauhtemoc...","[{'label': 'display', 'lat': 19.41838565946042...",19.418386,-99.158744,NaN,06700,Distrito Federal,Cafebrería El Péndulo,v-1596152623,NaN
1,"[{'id': '4bf58dd8d48988d114951735', 'name': 'B...",False,4e14fcab1f6e29fb7d4c9225,"Hamburgo, 126",MX,Cuauhtemoc,México,Amberes,3430,"[Hamburgo, 126 (Amberes), 06600 Cuauhtémoc, Di...","[{'label': 'display', 'lat': 19.42602901489909...",19.426029,-99.165032,NaN,06600,Distrito Federal,Cafebrería El Péndulo,v-1596152623,NaN
2,"[{'id': '4bf58dd8d48988d114951735', 'name': 'B...",False,4b0586fef964a520cc7922e3,Nuevo León 115,MX,Cuauhtemoc,México,NaN,4859,"[Nuevo León 115, 06700 Cuauhtémoc, Distrito Fe...","[{'label': 'display', 'lat': 19.41041906215392...",19.410419,-99.172883,NaN,06700,Distrito Federal,Cafebrería El Péndulo,v-1596152623,88328888
3,"[{'id': '4bf58dd8d48988d114951735', 'name': 'B...",False,4b8aa44bf964a520f87632e3,Alejandro Dumas 81,MX,Ciudad de México,México,NaN,6596,"[Alejandro Dumas 81, 11560 Ciudad de México, D...","[{'label': 'display', 'lat': 19.43043300996013...",19.430433,-99.195950,NaN,11560,Distrito Federal,Cafebrería El Péndulo,v-1596152623,NaN
4,"[{'id': '4bf58dd8d48988d114951735', 'name': 'B...",False,4fe39c32e4b079c77b248c37,Centro Comercial Perisur 2º piso,MX,Coyoacán,México,Anillo Periferico Sur 4690,15534,[Centro Comercial Perisur 2º piso (Anillo Peri...,"[{'label': 'display', 'lat': 19.3042604605492,...",19.304260,-99.190668,NaN,NaN,Distrito Federal,Cafebrería El Péndulo,v-1596152623,NaN


### Data analysis

In [8]:
#Libraries for the map
#!conda install -c conda-forge folium=0.5.0 --yes
import folium

In [9]:
cafebrerias = dataframe.drop(['categories', 'hasPerk', 'id', 'location.distance', 'location.labeledLatLngs', 'venuePage.id','location.cc'], axis = 1)
cafebrerias = cafebrerias.drop(['referralId','location.neighborhood'], axis = 1)
cafebrerias = cafebrerias.drop(cafebrerias[cafebrerias['name']== 'Almacén Cafebreria El Péndulo'].index)
cafebrerias = cafebrerias.drop(cafebrerias[cafebrerias['name']== 'Corporativo Cafebrería El Péndulo'].index)

In [10]:
cafebrerias = cafebrerias.rename(columns = {'location.city' : 'Alcaldia', 'location.country' : 'country','location.crossStreet':'Street', 'location.address': 'Address', 'location.formattedAddress':'Full Address'})
cafebrerias.head()

,Address,Alcaldia,country,Street,Full Address,location.lat,location.lng,location.postalCode,location.state,name
0,Álvaro Obregón 86,Cuauhtemoc,México,Córdoba,"[Álvaro Obregón 86 (Córdoba), 06700 Cuauhtemoc...",19.418386,-99.158744,06700,Distrito Federal,Cafebrería El Péndulo
1,"Hamburgo, 126",Cuauhtemoc,México,Amberes,"[Hamburgo, 126 (Amberes), 06600 Cuauhtémoc, Di...",19.426029,-99.165032,06600,Distrito Federal,Cafebrería El Péndulo
2,Nuevo León 115,Cuauhtemoc,México,NaN,"[Nuevo León 115, 06700 Cuauhtémoc, Distrito Fe...",19.410419,-99.172883,06700,Distrito Federal,Cafebrería El Péndulo
3,Alejandro Dumas 81,Ciudad de México,México,NaN,"[Alejandro Dumas 81, 11560 Ciudad de México, D...",19.430433,-99.195950,11560,Distrito Federal,Cafebrería El Péndulo
4,Centro Comercial Perisur 2º piso,Coyoacán,México,Anillo Periferico Sur 4690,[Centro Comercial Perisur 2º piso (Anillo Peri...,19.304260,-99.190668,NaN,Distrito Federal,Cafebrería El Péndulo


In [11]:
mexico_map = folium.Map(location = [latitude, longitude], zoom_start = 12)

for lat, lng, borough, neighborhood in zip(cafebrerias['location.lat'], cafebrerias['location.lng'], cafebrerias['name'], cafebrerias['Alcaldia']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(mexico_map)
    
mexico_map

In [12]:
del_mayor = cafebrerias.groupby(['Alcaldia']).size()
del_mayor

Alcaldia
Ciudad de México    4
Coyoacán            1
Cuauhtemoc          3
Iztapalapa          1
dtype: int64

In [13]:
cafebrerias_1 = cafebrerias.drop(cafebrerias[cafebrerias['Alcaldia'] != 'Ciudad de México'].index)
cafebrerias_2 = cafebrerias.drop(cafebrerias[cafebrerias['Alcaldia'] != 'Cuauhtemoc'].index)

In [14]:
cafebrerias_df = pd.concat([cafebrerias_1, cafebrerias_2], axis=0)
cafebrerias_df

,Address,Alcaldia,country,Street,Full Address,location.lat,location.lng,location.postalCode,location.state,name
3,Alejandro Dumas 81,Ciudad de México,México,NaN,"[Alejandro Dumas 81, 11560 Ciudad de México, D...",19.430433,-99.195950,11560,Distrito Federal,Cafebrería El Péndulo
7,Alejandro Dumas 81,Ciudad de México,México,NaN,"[Alejandro Dumas 81, 11560 Ciudad de México, D...",19.430140,-99.195980,11560,Distrito Federal,Cafebrería el Péndulo
14,Cerro de las Torres #6 local I Col. Campestre ...,Ciudad de México,México,Tlalpan,[Cerro de las Torres #6 local I Col. Campestre...,19.340980,-99.143157,04200,Distrito Federal,El Exilio: Cafebrería
16,Centro Santa Fe,Ciudad de México,México,Vasco de Quiroga 3800,"[Centro Santa Fe (Vasco de Quiroga 3800), 0614...",19.360804,-99.274413,06140,Distrito Federal,El Péndulo
0,Álvaro Obregón 86,Cuauhtemoc,México,Córdoba,"[Álvaro Obregón 86 (Córdoba), 06700 Cuauhtemoc...",19.418386,-99.158744,06700,Distrito Federal,Cafebrería El Péndulo
1,"Hamburgo, 126",Cuauhtemoc,México,Amberes,"[Hamburgo, 126 (Amberes), 06600 Cuauhtémoc, Di...",19.426029,-99.165032,06600,Distrito Federal,Cafebrería El Péndulo
2,Nuevo León 115,Cuauhtemoc,México,NaN,"[Nuevo León 115, 06700 Cuauhtémoc, Distrito Fe...",19.410419,-99.172883,06700,Distrito Federal,Cafebrería El Péndulo


In [15]:
latitude_2 = cafebrerias_2['location.lat'].mean()
longitude_2 = cafebrerias_2['location.lng'].mean()

In [16]:
mexico_map_1 = folium.Map(location = [latitude, longitude], zoom_start = 12)

for lat, lng, borough, neighborhood in zip(cafebrerias_df['location.lat'], cafebrerias_df['location.lng'], cafebrerias_df['name'], cafebrerias_df['Alcaldia']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(mexico_map_1)

mexico_map_1

###### a filter is not accurate in 'Alcaldia' so we do not ocunt it

In [17]:
mexico_map_2 = folium.Map(location = [latitude_2, longitude_2], zoom_start = 15)

for lat, lng, borough, neighborhood in zip(cafebrerias_2['location.lat'], cafebrerias_2['location.lng'], cafebrerias_2['name'], cafebrerias_2['Alcaldia']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(mexico_map_2)

mexico_map_2

### Coclusions

#### First conclusion

The "Alcaldia" with the most "cafebrerias" is Cuauhtemoc.


#### Second conclusion

According to the Mexicos city report, that _alcaldia_ has the most restaurants in the city, libraries, and buildings of departments. But more than all, is not quite far or next to the city center. 